In [1]:
import os
import sys
os.environ["SPARK_HOME"] = "/usr/hdp/current/spark2-client"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
sys.path.insert(0, os.environ["PYLIB"] + "/py4j-0.10.4-src.zip")
sys.path.insert(0, os.environ["PYLIB"] + "/pyspark.zip")

In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
spark = SparkSession.builder\
        .appName("Catalyst Optimizer")\
        .master('local[*]')\
        .enableHiveSupport()\
        .config('spark.sql.warehouse.dir','hdfs://bigdata:8020/user/mahidharv/spark-warehouse')\
        .getOrCreate()

In [3]:
sc = spark.sparkContext

In [4]:
spark

In [5]:
from pyspark.sql import Column as col
from timeit import timeit

In [6]:
sc._conf.getAll()

[(u'spark.history.kerberos.keytab', u'none'),
 (u'spark.eventLog.enabled', u'true'),
 (u'spark.history.ui.port', u'18081'),
 (u'spark.driver.extraLibraryPath',
  u'/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64'),
 (u'spark.executor.id', u'driver'),
 (u'spark.yarn.queue', u'default'),
 (u'spark.executor.extraLibraryPath',
  u'/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64'),
 (u'spark.yarn.historyServer.address', u'b.insofe.edu.in:18081'),
 (u'spark.history.fs.logDirectory', u'hdfs:///spark2-history/'),
 (u'spark.driver.host', u'p.insofe.edu.in'),
 (u'spark.sql.catalogImplementation', u'hive'),
 (u'spark.history.kerberos.principal', u'none'),
 (u'spark.rdd.compress', u'True'),
 (u'spark.history.provider',
  u'org.apache.spark.deploy.history.FsHistoryProvider'),
 (u'spark.app.name', u'Catalyst Optimizer'),
 (u'spark.serializer.objectStreamReset', u'100'),
 (u'spark.master', u'local[*]

In [12]:
#%%timeit -n 10
df1 = spark.range(20000000).toDF("a")
df2 = spark.range(10000000).toDF("b")
result = df1.join(df2,df1.a == df2.b).groupby().count()
result.show()

+--------+
|   count|
+--------+
|10000000|
+--------+



In [11]:
result.explain(True)

== Parsed Logical Plan ==
Aggregate [count(1) AS count#55L]
+- Join Inner, (a#40L = b#44L)
   :- Project [id#38L AS a#40L]
   :  +- Range (0, 20000000, step=1, splits=Some(8))
   +- Project [id#42L AS b#44L]
      +- Range (0, 10000000, step=1, splits=Some(8))

== Analyzed Logical Plan ==
count: bigint
Aggregate [count(1) AS count#55L]
+- Join Inner, (a#40L = b#44L)
   :- Project [id#38L AS a#40L]
   :  +- Range (0, 20000000, step=1, splits=Some(8))
   +- Project [id#42L AS b#44L]
      +- Range (0, 10000000, step=1, splits=Some(8))

== Optimized Logical Plan ==
Aggregate [count(1) AS count#55L]
+- Project
   +- Join Inner, (a#40L = b#44L)
      :- Project [id#38L AS a#40L]
      :  +- Range (0, 20000000, step=1, splits=Some(8))
      +- Project [id#42L AS b#44L]
         +- Range (0, 10000000, step=1, splits=Some(8))

== Physical Plan ==
*(6) HashAggregate(keys=[], functions=[count(1)], output=[count#55L])
+- Exchange SinglePartition
   +- *(5) HashAggregate(keys=[], functions=[partia

In [11]:
spark.stop()

In [17]:
spark = SparkSession.builder\
        .appName("Catalyst Optimizer")\
        .master('local[*]')\
        .enableHiveSupport()\
        .config("spark.sql.cbo.enabled",False)\
        .getOrCreate()

In [18]:
sc._conf.getAll()

[(u'spark.history.kerberos.keytab', u'none'),
 (u'spark.eventLog.enabled', u'true'),
 (u'spark.history.ui.port', u'18081'),
 (u'spark.driver.extraLibraryPath',
  u'/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64'),
 (u'spark.executor.id', u'driver'),
 (u'spark.yarn.queue', u'default'),
 (u'spark.executor.extraLibraryPath',
  u'/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64'),
 (u'spark.yarn.historyServer.address', u'b.insofe.edu.in:18081'),
 (u'spark.history.fs.logDirectory', u'hdfs:///spark2-history/'),
 (u'spark.app.id', u'local-1546007039197'),
 (u'spark.sql.catalogImplementation', u'hive'),
 (u'spark.history.kerberos.principal', u'none'),
 (u'spark.rdd.compress', u'True'),
 (u'spark.history.provider',
  u'org.apache.spark.deploy.history.FsHistoryProvider'),
 (u'spark.app.name', u'Catalyst Optimizer'),
 (u'spark.serializer.objectStreamReset', u'100'),
 (u'spark.master', u'local[*]'

In [19]:
df1 = spark.range(20000000).toDF("a")
df2 = spark.range(10000000).toDF("b")
result = df1.join(df2,df1.a == df2.b).groupby().count()

In [20]:
result.explain(True)

== Parsed Logical Plan ==
Aggregate [count(1) AS count#59L]
+- Join Inner, (a#44L = b#48L)
   :- Project [id#42L AS a#44L]
   :  +- Range (0, 20000000, step=1, splits=Some(8))
   +- Project [id#46L AS b#48L]
      +- Range (0, 10000000, step=1, splits=Some(8))

== Analyzed Logical Plan ==
count: bigint
Aggregate [count(1) AS count#59L]
+- Join Inner, (a#44L = b#48L)
   :- Project [id#42L AS a#44L]
   :  +- Range (0, 20000000, step=1, splits=Some(8))
   +- Project [id#46L AS b#48L]
      +- Range (0, 10000000, step=1, splits=Some(8))

== Optimized Logical Plan ==
Aggregate [count(1) AS count#59L]
+- Project
   +- Join Inner, (a#44L = b#48L)
      :- Project [id#42L AS a#44L]
      :  +- Range (0, 20000000, step=1, splits=Some(8))
      +- Project [id#46L AS b#48L]
         +- Range (0, 10000000, step=1, splits=Some(8))

== Physical Plan ==
*(6) HashAggregate(keys=[], functions=[count(1)], output=[count#59L])
+- Exchange SinglePartition
   +- *(5) HashAggregate(keys=[], functions=[partia